# Predicting Medical Insurance Cost Using Regression

Health Insurance cost is a major economic concern for most countries and corporate organisations. To remain in business and to make profits. insurance companies want to predict the exact amount to be charged on individual clients. To do this effectively, the companies lok at the health risk inherent to each client. Most appropriately, lifestyle and pre-existing condition are used to gauge the inherent risk associated with each patient and then to correctly predict the health Insurance charges for the individual.
This study aims to use some of these individual parameters as the independent variables, X to proedict the cost of health insurance, y(dpendent variable) for a particular individual. 

The data used for analysis was obtined from Kaggle. Several factors affects the overall health risk of individuals. Most of those factors have to do with individual's lifestyle and present health condition. The dataset lists some of those features determining the health risks of individuals and we want to use those factors in predicting the cost of Medical insurance for the individual.  

The purpose of this resarch is to try prediction with diferent regression models on the dataset and come out with the best regression model that can correctly predict the cost of Medical Isurance using the available data. All analysis was performed with python 3 using the 'Spyder' environment.

Our Target variable for the Regression studies is a numerical variable detailing the health insurance cost for each observation parameters. After necessary processing, the data is split and used for analysis. Five models were used to run the analysis and the results were compared with each other to chose the ones with the highest model scores.

The analytics process and results are presented below;

In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\MAIN'

In [4]:
os.chdir('C:\\Users\\MAIN\\Desktop\\ML\\ML Project')

In [5]:
os.getcwd()

'C:\\Users\\MAIN\\Desktop\\ML\\ML Project'

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Data Preprocessing

In [7]:

df= pd.read_csv("insurance.csv")
df.describe()
#handle missing values
df.isnull().sum()
df.dropna(axis=0,how='any',inplace=True)

#check correlation with seaborn heatmap
import seaborn as sns
sns.set_style("whitegrid")
plt.figure(figsize=(40,40))
sns.heatmap(df.corr(), annot=True, cmap='Greens')

#define X,y
X=df.drop(['charges'],axis=1)
y=df.charges

#Normalise the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = (sc.fit_transform(X))
features=X.columns.values

sc_y = StandardScaler()
y_scaled = sc_y.fit_transform(y.reshape(len(y),1)).ravel()

The correlation matrix shown on the heatmap analysis shown that the most feature with a strong positive correlation with the insurance charges was the 'smoker' variable followed by the "bmi". Th RFE was run after that to decrease the number of variables in the X independent variable. This process increases the predictive ability of the model by increasing the Adjusted R2-value.

In [ ]:
####################........Linear Regression.........#########################################
#import learning model(Linear Regression)
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
from sklearn.feature_selection import RFE

#Run RFE and select best subset of X
estimator = LinearRegression() #use regression model for regression problem
list_r2=[]
max_r2 = 0
for i in range(1,len(X.iloc[0])+1):
    selector = RFE(estimator, i, step=1)
    selector = selector.fit(X_scaled, y_scaled)
    adj_r2 = 1 - ((len(X)-1)/(len(X)-i-1))*(1-selector.score(X_scaled, y_scaled))
    list_r2.append(adj_r2)# mse = 
    if max_r2 < adj_r2:
        sel_features = selector.support_
        max_r2 = adj_r2
        
X_sub = X_scaled[:,sel_features]

Linear Regression was the first model used and the main parameters for comparison were the Mean Squared Error and the R-squared value.

In [ ]:
#statistical summary of the model
import statsmodels.api as sm
X2=sm.add_constant(X_sub)  #investigate
model=sm.OLS(y,X2)
results=model.fit()
print(results.summary())

#split train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sub, y_scaled, test_size=0.3, random_state=0)

model1 = LinearRegression()
#Train our model
model1.fit(X_train,y_train)
#predict with the model
y_pred = model1.predict(X_test)
b0 =model1.intercept_
b1 =model1.coef_

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print('intercept & coefficients: \n', model1.intercept_, model1.coef_)
print('mean squared error (training):', mean_squared_error(y_train, model1.predict(X_train)))
print('mean squared error (testing):', mean_squared_error(y_test, y_pred))
print('R-squared score(training):', r2_score(y_train, model1.predict(X_train)))
print('R-squared score(testing):', r2_score(y_test, y_pred))

#Run K-Fold using R2
from statistics import mean, stdev
from sklearn.model_selection import KFold, cross_val_score

shuffle = KFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(model1, X_sub, y_scaled, cv=shuffle)
print(scores)

In [ ]:
#####################.......KNearestNeighbor Regression..................###################################
#import Learning model
from sklearn.neighbors import KNeighborsRegressor
Regressor = KNeighborsRegressor(n_neighbors=21, weights='distance', p=1) #by default p=2. tweak n_neighbords to check accuracy

#train classifier
Regressor.fit(X_train,y_train)

#predictions for test
y_pred = Regressor.predict(X_test)

#import performance measure tools
from sklearn.metrics import mean_squared_error
import math

mse = mean_squared_error(y_test,y_pred)
rmse = math.sqrt(mse)
print('R-squared score(training):', r2_score(y_train, Regressor.predict(X_train)))
print('R-squared score(testing):',r2_score(y_test, y_pred))

#split X and y into K-Folds
#Run K-Fold using R2
shuffle = KFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(model, X_sub, y_scaled, cv=shuffle)
print(scores)

In [ ]:
#####################..........Random Forest..........#####################################################

#import Learning Model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

model.fit(X_train,y_train)

model.score(X_test,y_test)

mse = mean_squared_error(y_test,y_pred)
rmse = math.sqrt(mse)
print('R-squared score(training):', r2_score(y_train, model.predict(X_train)))
print('R-squared score(testing):',r2_score(y_test, y_pred))

#Run K-Fold using R2
shuffle = KFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(model, X_sub, y_scaled, cv=shuffle)
print(scores)

#check Gridsearch for model2 = RandomForest
from sklearn.model_selection import GridSearchCV
param_dict= {'n_estimators':range(2,30), 'max_depth':range(1,30)}
model = GridSearchCV(model,param_dict)
model.fit(X_train,y_train)

model.score(X_test,y_test)
model.best_params_

In [ ]:
##############............AdaBoost Model.............#############################################################

#import learning model
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor()

model.fit(X_train,y_train)

model.score(X_test,y_test)

mse = mean_squared_error(y_test,y_pred)
rmse = math.sqrt(mse)
print('R-squared score(training):', r2_score(y_train, model.predict(X_train)))
print('R-squared score(testing):',r2_score(y_test, y_pred))

#split X and y into K-Folds
#Run K-Fold using R2
shuffle = KFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(model, X_sub, y_scaled, cv=shuffle)
print(scores)

#check Gridsearch for model3 = AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
param_dict= {'n_estimators':range(2,30)}
model = GridSearchCV(model,param_dict)
model.fit(X_train,y_train)

model.score(X_test,y_test)
model.best_params_

In [ ]:
###############.............SVR model...........####################################################################

from sklearn.svm import SVR
model = SVR(kernel='rbf')

model.fit(X_train,y_train)
y_pred = model.predict(X_test)

model.score(X_test,y_test)

mse = mean_squared_error(y_test,y_pred)
rmse = math.sqrt(mse)
print('R-squared score(training):', r2_score(y_train, model.predict(X_train)))
print('R-squared score(testing):',r2_score(y_test, y_pred))

#split X and y into K-Folds
#Run K-Fold using R2
shuffle = KFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(model, X_sub, y_scaled, cv=shuffle)
print(scores)

# Visualising the Training set results
from matplotlib.colors import ListedColormap
X_set, y_set = X_train, y_train
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, model.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('SVM (Training set)')
plt.xlabel('Factors')
plt.ylabel('Insurance Charges')
plt.legend()
plt.show()

# Visualising the Test set results
from matplotlib.colors import ListedColormap
X_set, y_set = X_test, y_test
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, model.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('SVM (Test set)')
plt.xlabel('Factors')
plt.ylabel('Insurance Charges')
plt.legend()
plt.show()

## Results Comparison

REGRESSION MODELS AND THEIR SCORES
            MODEL	MSE	    R2	    K-FOLD
Linear Regression	0.23	0.79	[ 0.79972962 0.77927394  0.65223099  0.75484223  0.71574224]
K Nearest Neighbor	0.17	0.84	[ 0.85891079  0.8514326   0.76125618  0.83682225  0.76758842]
Random Forest	    0.13	0.8798	[ 0.84467637  0.86627158  0.74810597  0.83381184  0.79090569]
AdaBoost 		            0.8618	[ 0.84355021  0.86542682  0.73502305  0.82376483  0.78491412]
Support Vector Reg	0.1308	0.8798	[ 0.89149575  0.86976228  0.78512325  0.86635013  0.78377819]

The results showed a high degree of prediction scores for the different models employed in the study as shown with the R2 value.
From the scores, The Support Vector Regression and Random Forest Classifier seem to be the best model for prediction, though the other ones gave a high degree of prediction scores too. This means that we can use the features provided to the health insurance 
cost of individuals with a high degree of certainty.

The Support Vector Regression is a regression model used for continous variables. Unlike other regression models, the SVR tries to fit the error within  a certain threshold.